# 📰 Building a news classifier with weak supervision

## TL;DR

In this tutorial, we will build a news classifier using rules and weak supervision: 

- 📰 For this example, we use the AG News dataset but you can follow this process to programmatically label any dataset.
- 🤿 The train split without labels is used to build a training set with rules, Rubrix and Snorkel's Label model.
- 🔧 The test set is used for evaluating our weak labels, label model and downstream news classifier.
- 🤯 We achieve 0.81 macro avg. f1-score without using a single example from the original dataset and using a pretty lightweight model (scikit-learn's `MultinomialNB`).

The following diagram shows the overall process for using Weak supervision with Rubrix:

![Labeling workflow](https://raw.githubusercontent.com/recognai/rubrix-materials/main/tutorials/weak_supervision/weak_supervision.svg "Labeling workflow")

## Introduction

> *Weak supervision is a branch of machine learning where noisy, limited, or imprecise sources are used to provide supervision signal for labeling large amounts of training data in a supervised learning setting. This approach alleviates the burden of obtaining hand-labeled data sets, which can be costly or impractical. Instead, inexpensive weak labels are employed with the understanding that they are imperfect, but can nonetheless be used to create a strong predictive model.* [[Wikipedia]](https://en.wikipedia.org/wiki/Weak_supervision)

For a broader introduction to weak supervision, as well as further references, we recommend the excellent [overview by Alex Ratner et al.](https://www.snorkel.org/blog/weak-supervision).

This tutorial aims to be a practical introduction to weak supervision and will walk you through its entire process.
First we will generate weak labels with *Rubrix*, combine these labels with *Snorkel*, and finally train a classifier with *Scikit Learn*.

## Setup

Rubrix, is a free and open-source tool to explore, annotate, and monitor data for NLP projects.

If you are new to Rubrix, check out the ⭐ [Github repository](https://github.com/recognai/rubrix).

You can install Rubrix on your local machine, on a server, or using a cloud provider. 
If you have not installed and launched Rubrix, check the [Setup and Installation guide](../getting_started/setup&installation.rst).

For this tutorial we also need some third party libraries that can be installed via pip:

In [ ]:
%pip install snorkel datasets sklearn -qqq

## 1. Load test and unlabelled datasets into Rubrix

First, let's download the `ag_news` data set and have a quick look at it.

In [ ]:
from datasets import load_dataset

# load our data
dataset = load_dataset("ag_news")

In [11]:
import pandas as pd

# quick look at our data
with pd.option_context('display.max_colwidth', None):
    display(dataset["test"].to_pandas().head())

,text,label
0,Fears for T N pension after talks Unions representing workers at Turner Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.,2
1,"The Race is On: Second Private Team Sets Launch Date for Human Spaceflight (SPACE.com) SPACE.com - TORONTO, Canada -- A second\team of rocketeers competing for the #36;10 million Ansari X Prize, a contest for\privately funded suborbital space flight, has officially announced the first\launch date for its manned rocket.",3
2,"Ky. Company Wins Grant to Study Peptides (AP) AP - A company founded by a chemistry researcher at the University of Louisville won a grant to develop a method of producing better peptides, which are short chains of amino acids, the building blocks of proteins.",3
3,"Prediction Unit Helps Forecast Wildfires (AP) AP - It's barely dawn when Mike Fitzpatrick starts his shift with a blur of colorful maps, figures and endless charts, but already he knows what the day will bring. Lightning will strike in places he expects. Winds will pick up, moist places will dry and flames will roar.",3
4,"Calif. Aims to Limit Farm-Related Smog (AP) AP - Southern California's smog-fighting agency went after emissions of the bovine variety Friday, adopting the nation's first rules to reduce air pollution from dairy cow manure.",3


Now we will log the test split of our data set to *Rubrix*, which we will be using for testing our label and downstream models.

In [ ]:
import rubrix as rb

# get the index to label mapping 
labels = dataset["test"].features["label"].names

# build our test records
records = [
    rb.TextClassificationRecord(
        inputs=record["text"],
        metadata={"split": "test"},
        annotation=labels[record["label"]]
    )
    for record in dataset["test"]
]

# log the records to Rubrix
rb.log(records, name="news")

In a second step we log the train split without labels.
Remember, our goal is to programmatically build a training set using rules and weak supervision.

In [ ]:
# build our training records without labels
records = [
    rb.TextClassificationRecord(
        inputs=record["text"],
        metadata={"split": "unlabelled"},
    )
    for record in dataset["train"]
]

# log the records to Rubrix
rb.log(records, name="news")

The result of the above is the following dataset in Rubrix, with **127,600 records** (120,000 unlabelled and 7,600 for testing). 

You can use the web app to find good rules for programmatic labeling!

![News dataste](img/news_tutorial.gif "Snorkel exploration with Rubrix")

## 2. Create rules and weak labels

We can now define some rules for each category. 
For this, you can use the expressive power of Elasticsearch's query string DSL:

In [14]:
from rubrix.labeling.text_classification import Rule

# define queries and patterns for each category (using ES DSL)
queries = [
  (["money", "financ*", "dollar*"], "Business"),
  (["war", "gov*", "minister*", "conflict"], "World"),
  (["footbal*", "sport*", "game", "play*"], "Sports"),
  (["sci*", "techno*", "computer*", "software", "web"], "Sci/Tech")
] 

# define rules
rules = [
    Rule(query=term, label=label)
    for terms,label in queries
    for term in terms
]

In [ ]:
from rubrix.labeling.text_classification import WeakLabels

# generate the weak labels 
weak_labels = WeakLabels(
    rules=rules, 
    dataset="news"
)

On our machine it took around 24 seconds to apply the rules and to generate weak labels for the 127,600 examples.

Typically, you want to iterate on the rules and check their statistics. 
For this, you can use `weak_labels.summary` method:

In [16]:
weak_labels.summary()

,polarity,coverage,overlaps,conflicts,correct,incorrect,precision
money,{Business},0.008276,0.002437,0.001936,30,37,0.447761
financ*,{Business},0.019655,0.005893,0.005188,80,55,0.592593
dollar*,{Business},0.016591,0.003542,0.002908,87,37,0.701613
war,{World},0.011779,0.003213,0.001348,75,26,0.742574
gov*,{World},0.045078,0.010878,0.006270,170,174,0.494186
minister*,{World},0.030031,0.007531,0.002821,193,22,0.897674
conflict,{World},0.003041,0.001003,0.000102,18,4,0.818182
footbal*,{Sports},0.013166,0.004945,0.000439,107,7,0.938596
sport*,{Sports},0.021191,0.007045,0.001223,139,23,0.858025
game,{Sports},0.038879,0.014083,0.002375,216,71,0.752613


From the above, we see that our rules cover around **30% of the original training set** with an **average precision of 0.72**. Our hope is that the label and downstream models will improve both the recall and the precision of the final classifier.

## 3. Denoise weak labels with Snorkel's Label Model

The goal at this step is to **denoise** the weak labels we've just created using rules. There are several approaches to this problem using different statistical methods.

In this tutorial, we're going to use Snorkel but you can actually use any other Label model or weak supervision method, such as FlyingSquid for example (see the [Weak supervision guide](../guides/weak-supervision.ipynb) for more details).
For convenience, Rubrix defines a simple wrapper over Snorkel's Label Model so it's easier to use with Rubrix weak labels and datasets:

In [17]:
from rubrix.labeling.text_classification import Snorkel

# create the label model
label_model = Snorkel(weak_labels)

# fit the model
label_model.fit()

# test it with labeled test set
label_model.score()

{'accuracy': 0.746092099704267}

## 3. Prepare our training set

Now, we already have a "denoised" training set, which we can prepare for training a downstream model. 
The label model predict returns `TextClassificationRecord` objects with the `predictions` from the label model. 

We can either refine and review these records using the Rubrix web app, use them as is, or filter them by score, for example.

In this case, we assume the predictions are precise enough and use them without any revision. 
Our training set has ~38,000 records, which corresponds to all records where the label model has not abstained.

In [20]:
# get records with the predictions from the label model
records = label_model.predict()

# build a simple dataframe with text and the prediction with the highest score
df_train = pd.DataFrame([
    {"text": record.inputs["text"], "label": label_model.weak_labels.label2int[record.prediction[0][0]]}
    for record in records
])

# quick look at our training data with the weak labels from our label model 
with pd.option_context('display.max_colwidth', None):
    display(df_train)

,text,label
0,"US Global's Holmes Mines Commodity Stocks NEW YORK (Reuters) - Frank Holmes would have loved to become a boxer, but he hung up his gloves at an early age and took a jab at money management instead.",0
1,"Giants #39; only hope is the future As darkness descended in the Valley of the Sun, the Giants flickering playoff hopes may have been all but extinguished in the cool desert air.",1
2,"Mourinho #39;s Chelsea Faces Porto in Champions League (Update1) Chelsea, soccer #39;s biggest-spending team, will play defending champion Porto in the group stage of European soccer #39;s Champions League, pitting Blues coach Jose Mourinho against the team he took to last year #39;s final.",1
3,"Fiserv in Talks for Australian Pact Fiserv Inc., a provider of information technology services to the financial industry, on Friday said it is negotiating a deal to manage a proposed check-processing joint venture with three of Australia #39;s largest banks.",2
4,"Militants kill 12 in J amp;K ahead of PMs visit Prime Minister Manmohan Singh will arrive in Srinagar tomorrow on the first leg of his two-day maiden visit to Jammu and Kashmir during which his approach would be of looking at the big picture rather than doling out packages, sources in the Prime Minister",3
...,...,...
38080,"The day Hardie #39;s sums didn #39;t add up DENNIS Cooper is too much of a soft-spoken gentleman to publicly express anger, but the professional conflict between him and Peter Macdonald goes back a long way to a day in May 2001.",3
38081,"Sony net profit more than doubles; Spider-Man 2 a boost (AFP) AFP - Global consumer electronics giant Sony said its interim net profit more than doubled to 76.5 billion yen (720 million dollars), thanks in part to its popular movie Spider-Man 2.",0
38082,Let Bob do his job: agent CHICAGO -- Be prepared for the worst. That could be the message NHL Players Association executive director Bob Goodenow conveys to player agents at a meeting today.,1
38083,"Virgin Group to Unveil Portable Music Player By MAY WONG SAN JOSE, Calif. (AP) -- The consumer electronics arm of the Virgin Group is introducing a new 5-gigabyte hard-disk portable music player, bringing a powerful brand name in music to the increasingly crowded product space...",1


In [21]:
# for the test set, we can retrieve the records with validated annotations (the original ag_news test set)
df_test = rb.load("news", query="status:Validated")

# transform data to match our training set format
df_test['text'] = df_test.inputs.transform(lambda r: r['text'])
df_test['annotation'] = df_test['annotation'].apply(
    lambda r: label_model.weak_labels.label2int[r]
)

## 4. Train a downstream model with scikit-learn

Now, let's train our final model using `scikit-learn`:

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# define our final classifier
classifier = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB())
])

# fit the classifier
classifier.fit(
    X=df_train.text.tolist(), 
    y=df_train.label.values
)

In [24]:
# compute the test accuracy
accuracy = classifier.score(
    X=df_test.text.tolist(), 
    y=label_model.weak_labels.annotation()
)

print(f"Test accuracy: {accuracy}")

Test accuracy: 0.8163157894736842


Not too bad! 🥳

We have achieved around **0.82 accuracy** without even using a single example from the original `ag_news` train set and with a small set of rules (less than 30). Also, we've largely improved over the 0.74 accuracy of our Label Model.

Finally, let's take a look at more detailed metrics:

In [25]:
from sklearn import metrics

# get predictions for the test set
predicted = classifier.predict(df_test.text.tolist())

print(metrics.classification_report(label_model.weak_labels.annotation(), predicted, target_names=labels))

              precision    recall  f1-score   support

       World       0.88      0.56      0.69      1900
      Sports       0.86      0.98      0.91      1900
    Business       0.76      0.83      0.80      1900
    Sci/Tech       0.79      0.89      0.84      1900

    accuracy                           0.82      7600
   macro avg       0.82      0.82      0.81      7600
weighted avg       0.82      0.82      0.81      7600



## Summary

In this tutorial, we saw how you can leverage weak supervision to quickly build up a large training data set, and use it for the training of a first lightweight model.

*Rubrix* is a very handy tool to start the weak supervision process by making it easy to find a good set of starting rules, and to reiterate on them dynamically.
Since *Rubrix* also provides built-in support for the most common label models, you can get from rules to weak labels in a few straight forward steps.
For more suggestions on how to leverage weak labels, you can checkout our [weak supervision guide](../guides/weak-supervision.ipynb) where we describe an [interesting approach](../guides/weak-supervision.ipynb#Joint-Model-with-Weasel) to jointly train the label and a transformers downstream model.

## Next steps

If you are interested in the topic of weak supervision check our [weak supervision guide](../guides/weak-supervision.ipynb).

### ⭐ Rubrix [Github repo](https://github.com/recognai/rubrix) to stay updated.

### 📚 [Rubrix documentation](https://docs.rubrix.ml) for more guides and tutorials.

### 🙋‍♀️ Join the Rubrix community on [Slack](https://bit.ly/3o0Pfyk)